<a href="https://colab.research.google.com/github/yananma/5_programs_per_day/blob/master/04251.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 3.3 线性回归的简洁实现

### 3.3.1 生成数据集

In [0]:
import torch
from torch import nn 
import numpy as np 
torch.manual_seed(1)

torch.set_default_tensor_type('torch.FloatTensor')

In [0]:
num_inputs = 2 
num_examples = 1000 
true_w = [2, -3.4]
true_b = 4.2 
features = torch.tensor(np.random.normal(0, 1, (num_examples, num_inputs)), dtype=torch.float)
labels = true_w[0] * features[:, 0] + true_w[1] * features[:, 1] + true_b 
labels += torch.tensor(np.random.normal(0, 0.01, size=labels.size()), dtype=torch.float)

### 3.3.2 读取数据集

In [0]:
import torch.utils.data as Data 

batch_size = 10 

dataset = Data.TensorDataset(features, labels)

data_iter = Data.DataLoader(
    dataset=dataset, 
    batch_size=batch_size, 
    shuffle=True, 
    num_workers=4, 
)

In [6]:
for X, y in data_iter:
    print(X, '\n', y)
    break 

tensor([[-0.1649, -1.7607],
        [ 1.6516, -0.2547],
        [-0.4610,  0.5266],
        [ 0.7176, -0.3589],
        [ 0.1484, -1.2508],
        [ 1.0260,  0.1790],
        [-0.5323,  0.1450],
        [-1.5016, -0.3048],
        [ 1.0703,  1.8395],
        [ 3.5733,  0.1565]]) 
 tensor([ 9.8718,  8.3559,  1.4954,  6.8732,  8.7421,  5.6415,  2.6326,  2.2218,
         0.0736, 10.8089])


### 3.3.3 定义模型

In [7]:
class LinearNet(nn.Module):
    def __init__(self, n_feature):
        super(LinearNet, self).__init__()
        self.linear = nn.Linear(n_feature, 1)   # 阿拉伯数字 1，输出 1 个。

    def forward(self, x):
        y = self.linear(x)
        return y 

net = LinearNet(num_inputs)
print(net)

LinearNet(
  (linear): Linear(in_features=2, out_features=1, bias=True)
)


In [9]:
# 上一个是内部实现，以这个为准
net = nn.Sequential(
    nn.Linear(num_inputs, 1)
)

print(net)
print(net[0])

Sequential(
  (0): Linear(in_features=2, out_features=1, bias=True)
)
Linear(in_features=2, out_features=1, bias=True)


In [10]:
for param in net.parameters():
    print(param)

Parameter containing:
tensor([[-0.3435, -0.1186]], requires_grad=True)
Parameter containing:
tensor([-0.5459], requires_grad=True)


### 3.3.4 初始化模型参数

In [11]:
from torch.nn import init

init.normal_(net[0].weight, mean=0.0, std=0.01)
init.constant_(net[0].bias, val=0.0)

Parameter containing:
tensor([0.], requires_grad=True)

In [12]:
for param in net.parameters():
    print(param)

Parameter containing:
tensor([[0.0207, 0.0033]], requires_grad=True)
Parameter containing:
tensor([0.], requires_grad=True)


### 3.3.5 定义损失函数

In [0]:
loss = nn.MSELoss()

### 3.3.6 定义优化算法

In [14]:
import torch.optim as optim  

optimizer = optim.SGD(net.parameters(), lr=0.03)
print(optimizer)

SGD (
Parameter Group 0
    dampening: 0
    lr: 0.03
    momentum: 0
    nesterov: False
    weight_decay: 0
)


In [16]:
num_epochs = 3 
for epoch in range(1, num_epochs + 1):
    for X, y in data_iter:
        output = net(X)
        l = loss(output, y.view(-1, 1))
        optimizer.zero_grad()
        l.backward()
        optimizer.step()
    print('epoch %d, loss: %f' %(epoch, l.item()))

epoch 1, loss: 0.000133
epoch 2, loss: 0.000072
epoch 3, loss: 0.000061


In [18]:
dense = net[0]
true_w, dense.weight.data

([2, -3.4], tensor([[ 2.0001, -3.4009]]))

In [20]:
true_b, dense.bias.data

(4.2, tensor([4.1997]))